# Algorithm comparison

The computations below compares fits to the same simulated data using two different methods to compute the fit residuals. The function `costfunlsq` computes $\mathbf{r}_\text{TLS}(\mathbf{\theta})$, defined in Eq. (29) of [Mohtashemi et al. (2021)](https://doi.org/10.1364/OE.417724). The function `costfuntls` computes the residual vector
$$\mathbf{r}(\mathbf{\theta}, \mathbf{\mu}) = [\mathbf{\delta}(\mathbf{\mu}), \mathbf{\epsilon}(\mathbf{\theta}, \mathbf{\mu})], $$
where
$$[\mathbf{\delta}(\mathbf{\mu})]_k = \frac{x_k - \mu_k}{\sigma_{xk}$$
and
$$[\mathbf{\epsilon}(\mathbf{\theta}, \mathbf{\mu})]_k = {\frac{y_k - f_k(\mathbf{\theta}, \mathbf{\mu})}{\sigma_{yk}}.$$

In [ ]:
import numpy as np
from numpy import pi
from numpy.fft import rfftfreq, rfft, irfft
from numpy.random import default_rng

import scipy.linalg as la
from scipy.optimize import least_squares
from scipy import signal
from scipy.stats import cumfreq, chi2, norm, probplot

from matplotlib import pyplot as plt
from matplotlib.figure import figaspect

from thztools import thzgen, noiseamp
from thztools.thztools import _costfuntls

## Simulate set of noisy waveforms

In [ ]:
n = 256  # Number of samples
ts = 0.05  # Sampling time [ps]
t0 = 2.5  # Peak pulse time [ps]
fs = 1 / ts  # Sampling frequency [THz]

sigma_alpha = 1e-4  # Additive noise amplitude [signal units]
sigma_beta = 1e-2  # Multiplicative noise amplitude [dimensionless]
sigma_tau = 1e-3  # Time base noise amplitude [ps]
sigma_parms = np.array([sigma_alpha, sigma_beta, sigma_tau])

seed = 0  # RNG seed
rng = default_rng(seed)

The transfer function model function `tf_model` returns
 $$ H(\mathbf{\theta}; \omega) = \theta_0\exp(-i\omega\theta_1),$$
 which rescales the input by $\theta_0$ and shifts it by $\theta_1$ (using the $+i\omega t$ sign convention). In the simulations, $\theta_0 = 1$ and $\theta_1 = 0$. The time-domain response function `td_model` applies this transfer function to a signal vector $x(t)$ for a given $\mathbf{\theta}$ and returns the resulting $y(t)$, as shown in the second cell for $\theta_0 = 0.5, \theta_1 = 1.0$.

In [ ]:
def tf_model(_p, _w):
    return _p[0] * np.exp(-1j * _w * _p[1])


f = rfftfreq(n, ts)


def td_model(_p, _x):
    _y = irfft(rfft(_x) * tf_model(_p, 2 * pi * f), n=n)
    return _y

In [ ]:
n_sim = 250
p_opt = np.array([0.5, 1])
n_p = p_opt.size

mu, t = thzgen(n, ts, t0)
psi = td_model(p_opt, mu)
sigma_mu = noiseamp(sigma_parms, mu, ts)
sigma_psi = noiseamp(sigma_parms, psi, ts)

x_noise = sigma_mu * rng.standard_normal((n_sim, n))
y_noise = sigma_psi * rng.standard_normal((n_sim, n))

x = mu - x_noise
y = psi - y_noise

In [ ]:
plt.plot(t, mu)
plt.plot(t, td_model(p_opt, mu))
plt.show()

## Fits with `costfuntls`

In [ ]:
%%time

p_tls = np.empty((n_p, n_sim))
mu_tls = np.empty((n, n_sim))
resnorm_tls = np.empty((n_sim,))
delta_tls = np.empty((n, n_sim))
epsilon_tls = np.empty((n, n_sim))
cv_tls = np.empty((n_p, n_p, n_sim))
sigma_tls = np.empty((n_p, n_sim))
for i in range(n_sim):
    sigma_x = noiseamp(sigma_parms, x[i, :], ts)
    sigma_y = noiseamp(sigma_parms, y[i, :], ts)

    def jac_fun(_x):
        p_est = _x[:n_p]
        mu_est = x[i, :] + _x[n_p:]
        jac_tl = np.zeros((n, n_p))
        jac_tr = np.diag(-1 / sigma_x)
        jac_bl = -(
            np.stack(
                [
                    td_model([1, p_est[1]], mu_est),
                    irfft(
                        rfft(mu_est)
                        * (-1j * 2 * pi * f * tf_model(p_est, 2 * pi * f)),
                        n=n,
                    ),
                ],
                axis=0,
            )
            / sigma_y
        ).T
        jac_br = -(
            la.circulant(td_model(p_est, signal.unit_impulse(n))).T / sigma_y
        ).T
        jac_tot = np.block([[jac_tl, jac_tr], [jac_bl, jac_br]])
        return jac_tot

    result = least_squares(
        lambda _p: _costfuntls(
            tf_model,
            _p[:2],
            x[i, :] + _p[2:],
            x[i, :],
            y[i, :],
            sigma_x,
            sigma_y,
            ts,
        ),
        np.concatenate((p_opt, np.zeros(n))),
        jac=jac_fun,
        method='lm',
        x_scale=np.concatenate((np.array([1, 1]), sigma_x)),
    )
    p_tls[:, i] = result.x[:n_p]
    mu_tls[:, i] = x[i, :] + result.x[n_p:]
    delta_tls[:, i] = result.fun[:n]
    epsilon_tls[:, i] = result.fun[n:]
    resnorm_tls[i] = 2 * result.cost
    r = np.linalg.qr(result.jac, mode='r')
    r_inv = np.linalg.inv(r)
    cv_tls[:, :, i] = (r_inv @ r_inv.T)[:n_p, :n_p]
    sigma_tls[:, i] = np.sqrt(np.diag(cv_tls[:, :, i]))
    if not result.success:
        print(f"Fit did not converge at i = {i}")

cf_tls = cumfreq(resnorm_tls, 1000, defaultreallimits=(0, 1000))
gof_tls = cf_tls.lowerlimit + np.linspace(
    0, cf_tls.binsize * cf_tls.cumcount.size, cf_tls.cumcount.size
)
ecdf_tls = cf_tls.cumcount / n_sim

In [ ]:
# Find the index of the median value
i_show = np.argsort(resnorm_tls)[len(resnorm_tls) // 2]
norm_res_eps = np.sqrt(512 / 254) * epsilon_tls[:, i_show]
norm_res_delta = np.sqrt(512 / 254) * delta_tls[:, i_show]
osm, osr = probplot(np.concatenate((norm_res_eps, norm_res_delta)), fit=False)

w, h = figaspect(0.45)
_, axd = plt.subplot_mosaic(
    [["CDF", "x-res"], ["CDF", "y-res"]], figsize=(w, h)
)

# Panel (a)
axd["CDF"].plot(
    gof_tls, ecdf_tls, 'k-', label=r'$S_\mathrm{TLS}$', clip_on=False
)
axd["CDF"].plot(
    gof_tls,
    chi2.cdf(gof_tls, n - n_p),
    '--',
    c='gray',
    label=r'$\chi^2$',
    clip_on=False,
)
axd["CDF"].plot(np.median(resnorm_tls), 0.5, 'rx')

axd["CDF"].legend(loc='upper right', frameon=False)
axd["CDF"].text(0.025, 0.95, '(a)', transform=axd["CDF"].transAxes)

axd["CDF"].set_xlim(0, 1000)
axd["CDF"].set_ylim(0, 1)

axd["CDF"].set_xticks([0, 500, 1000])
axd["CDF"].set_yticks(np.linspace(0, 1, 6, endpoint=True))

axd["CDF"].set_xlabel('GOF statistic')
axd["CDF"].set_ylabel('Cumulative probabilty')

# Inset to (a)
ins = axd["CDF"].inset_axes([0.5, 0.2, 0.45, 0.45])
ins.plot(osr, osm, 'k.', ms=2)
ins.plot([-3, 3], [-3, 3], '--', c='gray')
ins.grid()

ins.set_xlim(-5, 5)
ins.set_ylim(norm.ppf([0.0005, 0.9995]))

ins.set_xticks(np.arange(-4, 4.5, 2))
ins.set_yticks(norm.ppf([0.005, 0.1, 0.5, 0.9, 0.995]))

ins.set_yticklabels(['0.005', '0.1', '0.5', '0.9', '0.995'])

ins.set_xlabel('Normed residuals\n(x and y)')
ins.set_ylabel('Probability')

# Panel (b)
markerline, stemlines, baseline = axd["x-res"].stem(
    t, norm_res_delta, linefmt='k-', markerfmt='.'
)
markerline.set_markersize(2)
stemlines.set_linewidth(0.5)
baseline.set_linewidth(1)

axd["x-res"].text(0.025, 0.9, '(b)', transform=axd["x-res"].transAxes)

axd["x-res"].set_xlim(0, 10)
axd["x-res"].set_ylim(-3.5, 3.5)

axd["x-res"].set_xticklabels([])
axd["x-res"].set_ylabel('Normed x-residual')

# Panel (c)
markerline, stemlines, baseline = axd["y-res"].stem(
    t, norm_res_eps, linefmt='k-', markerfmt='.'
)
markerline.set_markersize(2)
stemlines.set_linewidth(0.5)
baseline.set_linewidth(1)

axd["y-res"].text(0.025, 0.9, '(c)', transform=axd["y-res"].transAxes)

axd["y-res"].set_xlim(0, 10)
axd["y-res"].set_ylim(-3.5, 3.5)

axd["y-res"].set_xlabel('Time (ps)')
axd["y-res"].set_ylabel('Normed y-residual')

plt.show()

In [ ]:
print(
    f"Amplitude parameter estimates: "
    f"{np.mean(p_tls[0, :]):.5f} ± {np.std(p_tls[0, :]):.5f}"
)
print(
    f"Delay parameter estimates:     "
    f"({np.mean(1e3 * p_tls[1, :]):.2f} ± {1e3 * np.std(p_tls[1, :]):.2f}) fs"
)
print(
    f"Mean uncertainty estimate for amplitude parameter: "
    f"{np.mean(np.sqrt(cv_tls[0, 0, :])):.5f}"
)
print(
    f"Mean uncertainty estimate for delay parameter:     "
    f"{1e3 * np.mean(np.sqrt(cv_tls[1, 1, :])):.2f} fs"
)
print(
    f"Standard deviation of the amplitude parameter uncertainty estimate: "
    f"{np.std(np.sqrt(cv_tls[0, 0, :])):.2g}"
)
print(
    f"Standard deviation of the delay parameter uncertainty estimate:     "
    f"{1e3 * np.std(np.sqrt(cv_tls[1, 1, :])):.2g} fs"
)

### Parameter estimate distributions

The cells below produce probability plots that show the parameter distributions in more detail.

In [ ]:
osm_a_tls, osr_a_tls = probplot(p_tls[0, :], fit=False)
osm_eta_tls, osr_eta_tls = probplot(p_tls[1, :], fit=False)

In [ ]:
w, h = figaspect(0.45)

_, axs = plt.subplots(1, 2, figsize=(w, h))

axs[0].plot((-4.5, 4.5), (-4.5, 4.5), c="gray")
axs[0].plot(
    (osr_a_tls - p_opt[0]) / np.std(osr_a_tls),
    osm_a_tls,
    'o',
    ms=3,
    label=r"TLS",
)

axs[0].set_xlim(-3, 3)
axs[0].set_ylim(-3, 3)

axs[0].set_xlabel(r"Expected quantiles of $\hat{A}$")
axs[0].set_ylabel(r"Observed values of $\hat{A}$")

axs[1].plot((-4.5, 4.5), (-4.5, 4.5), c="gray")
axs[1].plot(
    (osr_eta_tls - p_opt[1]) / np.std(osr_eta_tls),
    osm_eta_tls,
    'o',
    ms=3,
    label=r"TLS",
)

axs[1].set_xlim(-3, 3)
axs[1].set_ylim(-3, 3)

axs[1].set_xlabel(r"Expected quantiles of $\hat{\eta}$")
axs[1].set_ylabel(r"Observed values of $\hat{\eta}$")

plt.show()